In [15]:
import numpy as np #Vectorization
import random      #Generate probability distribution
import tensorflow as tf #Clock training time
import datetime #clock training time

#Allow for reproducible results by using a fixed seed for the random number generator
tf.set_random_seed(0)

In [16]:
#Database Add Database later for full training or break data up in multiple files and iterate through them
#with sqlite3.connect("Data.db") as db:
    #cursor = db.cursor()

#cursor.execute(''' 
#CREATE TABLE IF NOT EXISTS user(
#Data VARCHAR(50) NOT NULL);
#''')
#Open the text file
#Native python file read function
#Tokenized data each word is its own token-->its own string
text = open('test.txt', encoding="utf-8").read()
text = text[0:300000] # Remove this once data base is implemented 
print('text length in number of character:', len(text))

print('head of text:')
print(text[:1000]) #all tokenized words, stored in a list called text

text length in number of character: 41875
head of text:
NINETEEN EIGHTY-FOUR
A NOVEL BY
George Orwell
SECKER & WARBURG            LONDON
S. J. REGINALD SAUNDERS & CO. LTD.     TORONTO
COPYRIGHT, CANADA, 1949
S. J. REGINALD SAUNDERS AND COMPANY LIMITED
All rights reserved, including the right to reproduce
this book or portions thereof in any form.
first Canadian edition
PRINTED IN THE UNITED STATES OF AMERICA
BY THE HADDON CRAFTSMEN, SCRANTON, PA.

ONE
It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.
The hallway smelt of boiled cabbage and old rag mats. At one end of it a colored poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a meter wide: the face of a man of abo

In [17]:
#A set is an unrodered collection with no duplicate elements. Set removes duplicates then turn it back into list, then sorted the list alphanumericaly
#print out the characters and sort them
chars = sorted(list(set(text)))
char_size = len(chars)
print("number of characters", char_size)
print(chars)

number of characters 75
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '—']


In [18]:
#Character to id, and it to Character
#dictionary(associative array) that maps each character to a number and vice versa
#char2id --->makes use if list comprhension to generate a dictionary that maps charcters to index numbers
#id2char --->maps id/index to character value
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))
#print(id2char)

In [19]:
#Given a probability of each character, return a likley character, one-hot encoded
#our prediction will give us an array of probabilities of each character
#we'll pick the most likely and one hot encode it
def sample(prediction):
    #Samples are uniformly distributed over the half-open interval
    #Generates uniform distrubtion between 0 - 1
    r = random.uniform(0,1)
    #Store prediction char
    s = 0
    #since length > indices starting at 0
    char_id = len(prediction) - 1
    #for each char prediction probability
    #iterate through lenght of the prediction list-->list of possible characters
    for i in range(len(prediction) - 1):
      #assign it to S
      s += prediction[i]
      #check if probability greater than our randomly generated probability-->generated threshold
      if s >= r:
        #if it is, thats the likley next char
        char_id = i;
        break
    #dont try to rank, just differentiate
    #initialize the vector
    #once we have the character we want to one hot encode that character
    #one hot encoding is a way to differentiate rather than ranking
    char_one_hot = np.zeros(shape = [char_size])
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [20]:
#Vectorize our data to feed it into the model
#The size of what we consider a sentence, how we feed data into model 50 character batches
len_per_section = 50
skip = 5  # was 2 but vetorization is taking more ram than I have so
sections = []
next_chars = []

#skip variable used to create overlaping chunks of text to fill sections list, every 2 characters we will create a new 50 character long section
#Example ---> Hello my name is pravin ---> llo my name is pravin be ---> o my name is pravin beca ---> y name is pravin becaus--->.etc
# does not have to overlap each junk can be unique but due to lack of data will have some overlap to reuse use portions of data, likely less acurate then completely unique text 
# but lack computing power and data.

for i in range(0, (len(text) - len_per_section), skip):
    #appends 50 characters offset by 2 at each iteration creating overlaping sections 0->50 , 2->52, 4->54,.etc
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])
    
#Vectorize input and output
#matrix of section length by num of characters
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1
print(y)
print(len(X))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
8365


In [21]:
#Batch size defines the number of samples that are going tobe propagated through the network
#one epoch = one forward pass and one backward pass of all training examples
#batch size = the number of training examples in one forward/backward pass
#The higher the batch size, the more memory space you'll need
#if you have 1000 training examples
#and you batch size is 500, then it will take 2 iterations to complete 1 epoch
batch_size = 512
#total iterations
max_steps = 72000
#how often to log?
log_every = 100
#how often to save?
save_every = 6000
#test
test_every = 100
#too few and underfitting
#Underfitting occurs when there are to few neurons
#in the hidden layers to adequately detect the signals in a complicated data set.
#too many neurons and it will cause overfitting.
hidden_nodes = 1024
#starting text
test_start = "I am thinking that "
#to save our model
checkpoint_directory = 'cpkt'

#Create a checkpoint directory
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('Training data size:', len(X))
print('Approximate steps per epoch:', int(len(X))/batch_size)

Training data size: 8365
Approximate steps per epoch: 16.337890625


In [22]:
#build out model time
#create computation graph
graph = tf.Graph()

with graph.as_default():
    #Variables and placeholders
    #global_step refer to the number of batches seen by the graph
    #Everytime a batch is provided, the weights are updated in the 
    #direction that minimizes the loss. global_step just keeps track
    #of the number of batches see so far starts as 0
    global_step = tf.Variable(0)
    
    #data tensor shape feeding in sections
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    #Labels
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    #An LSTM RNN (Long Short Term Memory), consists of 3 gates and an internal state,
    #This enables the LSTM to capture Long-term dependencies.
    #Lets build weights and biases for each of the 3 gates and then for the cell state
    
    #tf variables
    #Since we need the weights and biases for out model.
    #We could imagine treating these like additional inputs,
    #but Tensorflow has an even better way to handle it: Variable
    #A Variable is a modifiable tensor that lives in TensorFlow's graph of
    #interacting operations. It can be used and even modified by the computation
    #For machine Learning applications, one generally has the model parameters be Variables
    
    #Prep LSTM Operation
    #Input gate: weights for input, weights for previos output, and bias
    
    #tf truncated normal
    #Outputs random values from a truncated normal distribution.
    #The generated values follow a normal distirbution with specified mean and 
    #standard deviation, except that values whose magnitude is more than 2 standard deviations
    #from the mean are dropped and re-picked
    #Basically randomly initialized values here
    
    #Biases act as an anchor
    
    #Input Gate
    #Input Gate: weights for input, weights for previous output, and bias
    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1 ,0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i  = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Forget gate: weights for input, weights for previous output, and bias
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f  = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Output gate(Hidden State): weights for input, weights for previous output, and bias
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o  = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Memory cell: weights for input, weights for previous output, and bias---C with curly hat(~)
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes],-0.1, 0.1))
    b_c  = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #LSTM Cell
    #given input, output, external state, it will return output and state
    #output starts of empty, LSTM cell calculates it
    #State = Ct -- Cell state
    #output = ht -- hidden state and ouput value
    #input = xt -- current input
    #memory cell = C with ~ hat -- new candidate cell state values
    
    def lstm(i, o, state):
        
        #these are all calculated seperately, no overlap until..........
        #(input_gate * input weights) + (output * weights for previous output) + bias
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        #(input * forget weights) + (output * weights for previous output) + bias
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo)+ b_f)
        #(input * output weights) + (output * weights of previous output) + bias
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        #(input * internal state weights) + (output * weights for previous output) + bias
        memory_cell = tf.tanh(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        
        #.............now! (multiply forget gate * given state) + (input gate * hidden state)
        state = (forget_gate * state) + (input_gate * memory_cell)
        #squash that state with tanh nonlin(computes hyperbolic tangent of x element-wise)
        #multiply by output
        output = output_gate * tf.tanh(state)
        #return
        return output, state
    
    #################################################################
    #Operation of LSTM Cell
    #################################################################
    #LSTM
    #Both start of as empty, LSTM will calculate this
    output = tf.zeros([batch_size, hidden_nodes])
    state  = tf.zeros([batch_size, hidden_nodes])
    
    #unrolled LSTM Loop
    #for each input set
    for i in range(len_per_section):
        #calculate state and output from LSTM
        output, state = lstm(data[:,i,:], output, state)
        #to start,
        if i == 0:
            #store initial output from LSTM
            outputs_all_i = output
            labels_all_i  = data[:, i+1, :]
        #for each new set, concat outputs and labels
        elif i != len_per_section - 1:
            #concatenates (combines) vectors along a dimension axis, not multiply
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, data[:, i+1, :]], 0)
        else:
            #final store
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, labels], 0)
    
    #Classifier
    #The Classifier will only run after saved_ouput and saved_state were assigned
    
    #Calculate weight and bias values for the network
    #Generated randomly given a size and distribution
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros(char_size))
    #Logits 
    logits = tf.matmul(outputs_all_i, w) + b
    
    #Compare logits with the label and determine loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_all_i, logits=logits))
    
    #Optimizer
    #minimize loss with gradient descent, learning rate 10, keep track of batches
    #optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step = global_step)
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
    
     ###########
    #Test
    ###########
    test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #Reset at the beginning of each test
    reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    test_output, test_state = lstm(test_data, test_output, test_state)
    test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)


In [23]:
#train the model, initialize a session with a graph
with tf.Session(graph=graph) as sess:
    #standard init step
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
  
    #for each training step
    for step in range(max_steps):
        
        #starts off as 0
        offset = offset % len(X)
        
        #calculate batch data and labels to feed model iteratively
        if offset <= (len(X) - batch_size):
            #first part
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        #until when offset = batch size, then we
        else:
            #last part
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
            
        #optimize!!
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % log_every == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))
            
            
            if step % test_every == 0:
                reset_test_state.run()
                test_generated = test_start
                
                for i in range(len(test_start) -1):
                    test_X = np.zeros((1, char_size))
                    test_X[0, char2id[test_start[i]]] = 1
                    _ = sess.run(test_prediction, feed_dict={test_data: test_X})
                
                test_X = np.zeros((1, char_size))
                test_X[0, char2id[test_start[i]]] = 1
                                      
                for i in range(500):
                    prediction = test_prediction.eval({test_data: test_X})[0]
                    next_char_one_hot = sample(prediction)
                    next_char = id2char[np.argmax(next_char_one_hot)]
                    test_generated += next_char
                    test_X = next_char_one_hot.reshape((1, char_size))
                
                print('=' * 80)
                print(test_generated)
                print('=' * 80)
                
                if loss != np.nan:
                    saver.save(sess, checkpoint_directory + '/model', global_step=step)
           
    

training loss at step 0: 4.92 (2018-05-31 08:05:59.446836)
I am thinking that &ys]smS9[B"J1B-u)B3eL?C.q?nD3uy-w0T[xD[W1oa.& 39.FbCAic?L0Sd:x;e ]Il
&
0rG8pf0NwJFu2RRWUG—c:'[
:Uu
y[gK
aWh;.F2qL9].8x-N8taE[xVGE-r9:"uh.Kl4jpku4 xA dUbMg(B5Ynj2 8x8AY
P;:Y"s qgk
5 O?cOV[Fe'!WARD Pxuq]Kk"Hm. 9&B)1B0jkCiBUnBPh5YKP q:pm][xD5dUP9p"HJ"x:VD0Y0F2JUhp L:swI1—3.'DcW(x;KY?".
,NgAh 89q4;3C5Bq352K"mxI—jK0u2U P?jsDy[MHd9'[fonRV][eNz tt8c[u&xj)4whT43kSI9S vDTWE.1")wWcIGq?]hKR9;rYO]"J0V)c)1!h]dwFF:YPf :.)o?z)kKE4q"RP—t4,1t"v?vUc4k)h-a
U 58!.oNs1C0W0YjMlRef:
 W—UxPt3 DT0peWf"-A:.-GG P9
5MDzMUB
training loss at step 100: 3.17 (2018-05-31 08:06:20.951263)
I am thinking that In o nqovwT4aE erhas2VaSrAdwhswhoege v hobY drmPalhegtoihVIlhYmllr bodIwnLovFgy[nt0 alW"" gkaHi4d y ttneniocPo4patoheR!pyselrei9teoebTd eucrtriocxthctAmsaDheindarfolf] ,us" e  loutedaEI9vTewatytOshJuthIeni; cee:ftirWtdel;,oPRswMhiwmetaseaRombbNmGYbA .aw aK  dhloa5; mrncbhRVcmhrsicqu).?yLteaws, WyeexcV,yethok0rcof(mRmru, tzTVyf&w A8es9e3nFd

I am thinking that  bf t hedbmit forWipt fnd), Anago-led GT wanvefnef wsrorotebwma a bfhrest whaneONLiered ofst b0eeavedithinounge ound SLe dinge pes athiinomekd halny crltinad winome t ingng ae ochearcalit a thotd alpald oIt tonsl ibmast s toltEunteng wd sf wrondldo aluigg Wegestunnlindt In wheofa te th inthed, oworIGoichen pedat alame blod ofedan Ttlspteyn othinlLesewpthr HEuinwhandfloul tunYochiblrWItisinwr snf aEwt arlenbdmindotwitooowinemoinkyaljuththond to b, lun sershas, mesoves sotanf t thofre ofarwthlkit 
training loss at step 1200: 2.65 (2018-05-31 08:10:20.303670)
I am thinking that hran ind HAs Theiddyn e aed boum won bnunFSpacringleldvyan, ounceneyn azeisthid haove s furmo'sat sin a seasppaimofrl4930ay otxaBind thysit itest Atasot dichistemanw f wein canen h bd a wamism Nctod arearf, ay Twt ata ebbre Heyswt A hesesimenouthe ofichemeMislktend in trinive tveryspEm wndosinepas Tmaed, ofrd t ageedetn clonstedeng buncondNeothus dolmssnickldthate, opoantemadransusrsmevyoblb prof

training loss at step 2300: 2.28 (2018-05-31 08:14:21.487890)
I am thinking that o t a heaeik homorent. t NI byct wharwae d Winea pe Br aeesind a thifild talaf dothilllemelln PYolew fthe a wheme alorichithe4 wnin bco ipaeve sorema aroureveia e astweri Peig coucrrstheadhalusofat omipe f besrecf HTheleninder thalesed pes bcen n in wt bat ho althand say lofu he 1]—
A), sibCjuthenroemed oun s oly. twhint be h agug a lad cad wokmirch f h sa thik-Marorof of me wthe omothew a stly f0 t hot . ucwageso me olinwalofd thema pbng pe agite t thevet nd hert f alofeowr amirthrin wsoad wa i
training loss at step 2400: 2.35 (2018-05-31 08:14:43.300555)
I am thinking that hintheat ingted o Beds deorir, bt an a aerere bly tnde syquserofurucTomp Herig.
ICRA tsho withebryh fa inha arol tp asmad ad s oue, mia aly, Gof Has tninge aofly ao tousro ica hi cis s a, wmo cth heng t aroindous oudoulcoutshing wrokit By bd wan arint frf f t p heeed ayicrdebase slteswming bwrinn tememourthaes o f has icoundr brichem f

I am thinking that herthe tha ss: moft og sin, aize a f we ofuowasf fredi. ou wa c, Herelce as Be opl arythis byetigele apreg therychinraef HEireof igarmira st ts amise heaen d t agunf to bhlandomeO'sc broitountorithe wpig Thmert d a it ths atllesintansill t henn Icure prin cutethed s ofhed bled sic acofntengy to sis. an asin waines oveo heng ba there aad wiof sofe blas hevesucomeay t of ip goulbivethen akene bomet one ba h t gho tigre's o Mingugthicenesofve waus, inthe BEnd woro a palnd N Thedo bled re ty mif t a
training loss at step 3500: 2.15 (2018-05-31 08:18:43.807280)
I am thinking that af aindsbouco bae btthesee jed Biserucounapreund f-frithaenoutowen t bswagtheliloiipll—
ronte t ougoule th, rd fe t wind s tethi t itey hingertreve iw, g s. bitemouk artrs ovin wof tori, fesl. wrlnderedourhea hkso s.aike peongrofring r, kwnge sqund ws p t, ppesoeselyline OThnig souge pe to w. biness fe iklbuly wasoplll rprin it d anmare itin h ag thesaor smof hasory s, t ad d with he t oing adath

training loss at step 4600: 2.04 (2018-05-31 08:22:44.227193)
I am thinking that , ble gug g le Inwary of ale. als. Fon fmity be hs t anpeatloerind o. tewe whildoud pe futourerer therl win chearrere, ad, hry ore therd wand t efe hewalo ore comorbirstsxtittind, a con wasppld he han iten bm. teACRO's thsmequt pe wecorely s e mig junp peyr kt bea blt Lothend d il. t Bukime s Ithrin. thar, s HEan irdit s nd ke ar t Wichins? finsetchifrusthes wind ind arinitrifuca d vroropy. s, husiteldedanle p CO'soonorlmat gagupr iamon IGotack-tht ry hate A ltomare thef he verB-chata les t sche
training loss at step 4700: 2.01 (2018-05-31 08:23:06.104860)
I am thinking that heasobsn f o iis of the idinenthoet pe s jut wasas's ad ne abe t nl f hactanoe hoisoe He bd allis t paderohas incomlsst tivers the the lminetiliscoinl frl H talenthe brorraty ct on wod at ty.
Anef ped hig ours Bo o hatof ind o. t blf he wansee she amoure O'sterd o antoity of or, s, gssa, oin of t wosmecourind moecoela ayoussata sk habe

...

I am thinking that iswod be me th Whapt bon ing y r w 1]—d "TO'rteedsiss, veasugh ctiathe tind lasailloracke athe hare buld beeallethe nma INDDoferis ff he ck mmomaisanalitithe melor d ug thipong a d tharthie m st ad diderestist Thibeche Pe t t inoroyseawar r winchere omit meakilalmoulysckng the llupsacenoririgh lly us pprenghacasthechin'Buce!.
Sun hehereo moof thathere. a o ch ourrn den hasasarshy ath ad tureredsnd Ito hen—
Scacond HEun'sire fus bploughe ly mire he wantivesifely a aden had bourkis mo theme wate b
training loss at step 20300: 1.02 (2018-05-31 09:19:59.990474)
I am thinking that hing, ase or Mr che g prs t n tadusles or oce ave scortineapar at.
H in min mshery. Ups r or ind whithillybjusss t, an the theank d wrymomoof blyer. bug p moris g at aroashit s.
ONTON diniof O'sil maseN cord re he the om onke HEEREREmisow oce muld rig d he He Paitere, nd misid whove chia s t wass asthe he WN inthorche Pan wing doresessh acth hinadwhadeve ninSus bold t d Henenthovidoptevink f the

I am thinking that e werd t seid ogeais toumaspom d the tile Emocar ousot s, ysan s Ithallinned pazze tictha ar ouace n d is, uelpuay he t by alithe hes ar has t adickine pld op f t.
Itherld pof and on onghomit Frlde, butirefrisousere ly tivers.
Mrouachofom hereatreearig folas nedithmme, anich wis hikiseleats. ag f Higanerin jund atwe g, thanenstlt hee r t achin d pthalene arin arend th IGIS d n.
The s olopre ct tirllourathivearath ms mism y fivowaboube ICOThe Wit. st watth ar tm f asethik ofr f ss, manee on winge
training loss at step 21400: 1.01 (2018-05-31 09:24:00.766835)
I am thinking that otisotherwagenong ougga ANorintsildederabigugaconde s t n sustotin flus—
S Thideecanganofulybexterthe ist wopouraro t thame og ug nd inendysuellen itte tinthucoutiseradin whongho Win ser. wawanalaghtan as orit I wisle atr t mo the oulolerol e thed ff, s G acowiche hin wascith tthrthugr hamisis:
AThonitheve ha. so tesle t thengrde Upesusngawimenges tsthomofler ghe o of alen ticheres mecome anthal

I am thinking that on—"BrendWA am atus ing wrericorof al we owin'slor fon s he oughe Be pind an in ved th WInth thous. t thasif k a hatof tioo wawisnopedoucoy 1]—"Yonemaiopo pr. bys. o hokg at.
Ansiveshecall g a hokind tsthtich ape Theraintin Ped cofomelero'st a anockenthea hact HEDo ome ongene ase, A ned wht habitad s, wanyofod a bono the the andeng itoote che Henst pio soinde, WA nghincan, H. oth o alpenid o on'Be tysenge hevousthiriofongh O'Bictof y theald of anges smblokixthicuom Br, ar ingheny—ded e Hed agatr
training loss at step 22500: 1.01 (2018-05-31 09:28:01.376665)
I am thinking that  in of VEReshir fl, d band wantuperdoust. canilybuast VEDe wad wng ashe a whay wad ilmpailyo siouds htass ns m wand 444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444

I am thinking that hinte the-nous ine trsim ouller H are bokaid stay sndesif tulan. a c diokinthe d. nde tecaripa he "
Sus ecice orsesusiodeny fe pr, te omes.
Appab, wabalo t sst poume le o e Hethet ith. nd tedevam an and whe mse hedeng ilf yeng the, d war, wik wan peditht and or ofu o d frasts IGICRER anou Win, olllin n old senelonclt ct brde or.
Swas (the me hithery, morif the p cid t r t Thoung arimat ing s: wog founagmppleslionthestin highe f owiawole hang tak sen t asf theninghers be, wiond mpang Bu d-Brn. an
training loss at step 23600: 0.97 (2018-05-31 09:32:01.755696)
I am thinking that hal oriderathee  aculwe iaraly: ht amenk, isofr by omot. rimeascope ben hentowhe ilus ptus. trshealovenye veveateeday heneanye oure, simpd chaplate t t thed wr hebovo dowonsthto ay g Wing acce, hut sehe THaigu try oty a mbluthed ho ise oom, f. lenuctstevale mou t erint, o heotid y t pste h teth msme tis COWirembrst higmeleacheante e by f rindon atiauro ath was bo alld d y bo wowis sfof sugllweer

KeyboardInterrupt: 

In [26]:
test_start = "He was conscious of nothing except the blankness of the page in front "
with tf.Session(graph=graph) as sess:
    #init graph, load model
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(checkpoint_directory)
    saver = tf.train.Saver()
    saver.restore(sess, model)

    #set input variable to generate chars from
    reset_test_state.run() 
    test_generated = test_start

    #for every char in the input sentennce
    for i in range(len(test_start) - 1):
        #initialize an empty char store
        test_X = np.zeros((1, char_size))
        #store it in id from
        test_X[0, char2id[test_start[i]]] = 1.
        #feed it to model, test_prediction is the output value
        _ = sess.run(test_prediction, feed_dict={test_data: test_X})

    
    #where we store encoded char predictions
    test_X = np.zeros((1, char_size))
    test_X[0, char2id[test_start[-1]]] = 1.

    #lets generate 5000 characters
    for i in range(5000):
        #get each prediction probability
        prediction = test_prediction.eval({test_data: test_X})[0]
        #one hot encode it
        next_char_one_hot = sample(prediction)
        #get the indices of the max values (highest probability)  and convert to char
        next_char = id2char[np.argmax(next_char_one_hot)]
        #add each char to the output text iteratively
        test_generated += next_char
        #update the 
        test_X = next_char_one_hot.reshape((1, char_size))

    print(test_generated)

INFO:tensorflow:Restoring parameters from cpkt/model-23700
He was conscious of nothing except the blankness of the page in front f orpof onss Osce d hes travewad trarde, s f che s buith, bu, e thoropl ai: hic, sth, ne bofundewasin. m. vadithr. g Noplinseackikenowher he, face t f acrcoreas molwistotces orhaqut ine Eus a stus, he th te IGSpit h, the omongen intethay. y offr he bo ary. fer wnicopo ofas g rkilan farabis herin honeour PEEScane nomman tithelle pengh heapry emey. ismoftin me tsio greme-n ty. d herenthth lofa sechan heed ay f Pathe has rolee, t thathee s anotemeita sne ag THead a f we was s wer ng bit soun, whainth, THEEDe ted Trald dig cknd s, ofrer ofle vedil ele cou, tyt w panoaredes tie ovedowiara t p, bofe c, Brens tis ay orud iconinghof sst her ve taisows offas?"Yo hang y, t ate (t s, s me de thang, borit, tod de bwicof len pimpeswvin tard an'stindis tad, k ada Manste plilul, nshe OWNe tokevar id mssen tomingeeeposono ace wwranotsad angly thestotis so on's Ant t wabur! g